# Computer Vision: Final Project

Submitted by: [ **Andrey Gresko, 334041076 | Amit Levy, 307928010** ]

In [1]:
import cv2
import numpy as np

In [2]:
# Parameters for Shi-Tomasi corner detection
feature_params = dict(maxCorners = 300, qualityLevel = 0.1, minDistance = 2, blockSize = 7)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize = (15,15), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [3]:
# The video feed is read in as a VideoCapture object
cap = cv2.VideoCapture("maze.mp4")

In [4]:
# Variable for color to draw optical flow track
color = 255 # white 
ret, first_frame = cap.read()
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
prev = cv2.goodFeaturesToTrack(prev_gray, mask = None, **feature_params)

In [5]:
# Creates an image filled with zero intensities with the same dimensions as the frame - for later drawing purposes
mask = np.zeros_like(prev_gray)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # total frame amount
frame_cnt = 0

while (cap.isOpened() and frame_cnt < length - 25): # -25 frames because video has ~0.5 fade out in the end
    # ret = a boolean return value from getting the frame, frame = the current frame being projected in the video
    ret, frame = cap.read()
    if frame is None or ret == False:
        break
    frame_cnt += 1
    enter_flag = True
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    next, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev, None, **lk_params)
    # Selects good feature points for previous position
    good_old = prev[status == 1]
    # Selects good feature points for next position
    good_new = next[status == 1]
    # Draws the optical flow tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        # Returns a contiguous flattened array as (x, y) coordinates for new point
        a, b = new.ravel()
        # Returns a contiguous flattened array as (x, y) coordinates for old point
        c, d = old.ravel()
        speed_vector = np.sqrt((a - c)**2 + (b - d)**2)
        if speed_vector > 1:
            enter_flag = False
            # Draws filled circle (thickness of -1) at new position with green color and radius of 3
            cv2.circle(mask, (a, b), 1, color, -1)
    if enter_flag == True: # draw path if the points were bad (refresh points to track)
        prev = cv2.goodFeaturesToTrack(prev_gray, mask = None, **feature_params)
    else:
        prev = good_new.reshape(-1, 1, 2)
    # Overlays the optical flow tracks on the original frame
    mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    mask_bgr = cv2.GaussianBlur(mask_bgr, (0,0), 3)
    mask_bgr = cv2.applyColorMap(mask_bgr, cv2.COLORMAP_HOT)
    output = cv2.add(frame, mask_bgr)
    cv2.imwrite("./heat_map.jpg", mask_bgr) # save heat map as an image
    cv2.imshow('Heat Map', mask_bgr) # show heat map video stream
    cv2.imwrite("./opt_flow_hotpath.jpg", output) # save motion path as an image
    cv2.imshow("Optical Flow", output) # show motion path
    prev_gray = gray
    if cv2.waitKey(1) & 0xFF == 27:
        break

# The following frees up resources and closes all windows
cap.release()
cv2.destroyAllWindows()